In [100]:
import os
import io
import numpy as np
import math
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from PIL import Image
from ipywidgets import widgets
from os import path

import zipcodes

import forge
from data import data
from data.image import coloring


In [97]:
DEBUG = False


def show(label, img=None):
  if img is None:
    img = label
  else:
    print(label)

  height, width = img.shape[:2]
  max_dim = max(width, height)
  if max_dim < 200:
    width = round(width * 200 / max_dim)
    height = round(height * 200 / max_dim)
  f = io.BytesIO()
  Image.fromarray(np.array(img, dtype=np.uint8)).save(f, 'png')
  # This method keeps data out of notebook source.
  widget = widgets.Image(
      value=f.getvalue(),
      width=width,
      height=height,
  )
  display(widget)



def load_zip():
  prefix = data.project_path('data/zip_line/')

  result = {}
  count = 0
  for root, dirs, files in os.walk(data.project_path('data/zip_line/')):
    city_state = root[len(prefix):].replace(os.sep, '').split(',')
    if not city_state:
      continue
    for file in files:
      if file in result:
        raise Exception('duplicate zipcode:', file)
      result[file] = tuple(city_state)
      count += 1
      if DEBUG: print('%s\t%s' % ('\t'.join(city_state), file))
    if DEBUG and count > 100:
      break

  return result


# are files all empty?
#def check_dirs()


def write_zip(result):
  with open(data.project_path('data/zip_line.txt'), 'w') as out:
    for zipcode, city_state in result.items():
      out.write('%s\t%s\n' % ('%s\t%s' % city_state, zipcode))

      
def write_translations(result):
  count = 0
  with open(data.project_path('data/zip_line_translated.csv'), 'w') as out:
    out.write('zip,correct,rotn,rotated\n')
    for zipcode, city_state in results.items():
      locations = zipcodes.matching(zipcode)
      if not locations:
        raise Exception('Not found:', zipcode, city_state)
      key = ''.join(city_state)
      for location in locations:
        reference = location['city'].replace(' ', '') + location['state'][-2:]
        if DEBUG: print(zipcode, reference, '+%d' % offset(reference, key), key)
        out.write('%s,%s,%s,%s\n' % (zipcode, reference, offset(reference, key), key))
        count += 1
      if DEBUG and count >= 20:
        break
      

def offset(s1, s2):
  if len(s1) != len(s2):
    return None
  items = iter((ord(c2) - ord(c1) + 26) % 26 for c1, c2 in zip(s1.lower(), s2.lower()))
  first = next(items, None)
  if all(x == first for x in items):
    return first
  return None


In [94]:
# results = load_zip()
# write_zip(results)


In [95]:
write_translations(results)

In [71]:
offset('RRUEJSZ', 'LloydMT')

20

In [72]:
offset('LloydMT', 'RRUEJSZ')

6

In [45]:
zipcodes.matching('59535')

[{'zip_code': '59535',
  'zip_code_type': 'STANDARD',
  'active': True,
  'city': 'Lloyd',
  'acceptable_cities': [],
  'unacceptable_cities': ['Chinook'],
  'state': 'MT',
  'county': 'Blaine County',
  'timezone': 'America/Denver',
  'area_codes': ['406'],
  'world_region': 'NA',
  'country': 'US',
  'lat': '48.0382',
  'long': '-109.2177'}]

In [78]:
for zipcode, city_state in results.items():
  locations = zipcodes.matching(zipcode)
  if not locations:
    raise Exception('Not found:', zipcode, city_state)
  key = ''.join(city_state)
  for location in locations:
    reference = location['city'].replace(' ', '') + location['state']
    print(zipcode, reference, '+%d' % offset(reference, key), key)
  

59535 LloydMT +6 RRUEJSZ
03777 OrfordNH +3 RUIRUGQK
88255 LocoHillsNM +6 RUIUNORRYTS
23404 LocustvilleVA +6 RUIAYZBORRKBG
36455 LockhartAL +6 RUIQNGXZGR
78644 LockhartTX +6 RUIQNGXZZD
70374 LockportLA +6 RUIQVUXZRG
52635 LockridgeIA +6 RUIQXOJMKOG
64654 LockSpringsMO +6 RUIQYVXOTMYSU
36865 LoachapokaAL +6 RUGINGVUQGGR
84758 OrdervilleUT +3 RUGHUYLOOHXW
90815 LongBeachCA +6 RUTMHKGINIG
90814 LongBeachCA +6 RUTMHKGINIG
90835 LongBeachCA +6 RUTMHKGINIG
90834 LongBeachCA +6 RUTMHKGINIG
75604 LongviewTX +6 RUTMBOKCZD
75615 LongviewTX +6 RUTMBOKCZD
61544 LondonMillsIL +6 RUTJUTSORRYOR
52755 LoneTreeIA +6 RUTKZXKKOG
73655 LoneWolfOK +6 RUTKCURLUQ
93545 LonePineCA +6 RUTKVOTKIG
81524 LomaCO +6 RUSGIU
56654 LomanMN +6 RUSGTST
61454 LomaxIL +6 RUSGDOR
65685 LouisburgMO +6 RUAOYHAXMSU
30434 LouisvilleGA +6 RUAOYBORRKMG
40294 LouisvilleKY +6 RUAOYBORRKQE
40205 LouisvilleKY +6 RUAOYBORRKQE
40255 LouisvilleKY +6 RUAOYBORRKQE
40285 LouisvilleKY +6 RUAOYBORRKQE
40215 LouisvilleKY +6 RUAOYBORRKQE
31814

In [120]:
SCALE = 10

img = np.zeros((360 * SCALE, 360 * SCALE, 3))
colors = list(coloring.colors(10))
for zipcode, city_state in results.items():
  locations = zipcodes.matching(zipcode)
  if not locations:
    raise Exception('Not found:', zipcode, city_state)
  key = ''.join(city_state)
  for location in locations:
    reference = location['city'].replace(' ', '') + location['state'][-2:]
    lat = 360 - int(float(location['lat']) * SCALE) + 180
    long = int(float(location['long']) * SCALE) + 180
    if offset(reference, key) is None:
      raise Exception('not matched', zipcode, city_state, reference)
    else:
      img[lat][long] = colors[offset(reference, key) - 1]

show(img)

Exception: ('not matched', '76127', ('REZEPEMVWXEXMSRNVF', 'XB'), 'NavalAirStation/JrbTX')

In [121]:
show(img)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x0e\x10\x00\x00\x0e\x10\x08\x02\x00\x00\x00\x92\xd88…

In [111]:
'NavalAirStation/JrbTX'[-2:]

'TX'